We are going to refactor the tables so all the data is in one table and all the metadata (about what was recorded) is in another table. This way the names of the tables won't be such a mess and we can process things easier through SQL logic.

As it currently is we have multiple data tables that look like this:

**<name=water_yeast_25uL_of_250mg_mL_yeast>**
| id | time | time_from_start | dissolved_oxygen | nanoamperes | temperature |
|:---|:-----|:----------------|:----------------|:------------|:------------|
| 1 | datetime | 00:01 | 8.80 | 50.0 | 20.0 |
|...|...|...|...|...|...|

We want to modify the database to have two tables. One that holds all the descriptive information of the study, like this:

**<name=dissolved_oxygen_study_table>**
| id | start_time | sample_name | solvent | hemoglobin_concentration_mg_mL | microsphere_concentration_uL_mL | yeast_stock_added_uL_mL | yeast_concentration_mg_mL |
|:---|:-----------|:------------|:--------|:-------------------------------|:--------------------------------|:-------------------------|:--------------------------|
| 1 | datetime | phantom1 | water | 1 | 0.5 | 0 | 0 |
| 2 | datetime | phantom2 | water | 1 | 0.5 | 100 | 250 |
|...|...|...|...|...|...|...|...|

And a second table that holds all the data from each sample and the reference to sample data like this:

**<name=dissolved_oxygen_records>**
| id | sample_name | sample_id | time | dissolved_oxygen | nanoamperes | temperature |
|:---|:------------|:----------|:-----|:-----------------|:------------|:------------|
| 1 | phantom1 | 1 | datetime | 8.00 | 50.00 | 20.0 |
| 2 | phantom1 | 1 | datetime | 7.99 | 49.97 | 20.2 |
|...|...|...|...|...|...|...|
| 100 | phantom2 | 2 | datetime | 8.30 | 51.2 | 19.7 |
| 101 | phantom2 | 2 | datetime | 8.27 | 50.9 | 19.9 |
|...|...|...|...|...|...|...|

Where the sample_name and sample_id columns for a foreign key pair that uniquely identifies a study row in the first table.

To do migrate the current tables to this format, we will iterate through each table, prompt input for the metadata fields, and then add all the data and metadat to new tables. Then we can confirm correct migration before dropping all the old tables.


In [1]:
import sqlite3
import tkinter as tk
from tkinter import filedialog, simpledialog

def get_metadata_from_user(table_name):
    """Prompt user for metadata fields based on the table name using a GUI."""
    metadata = {}
    root = tk.Tk()
    root.withdraw()

    metadata["sample_name"] = simpledialog.askstring("Metadata Input", f"Enter Sample Name for {table_name}:")
    metadata["solvent"] = simpledialog.askstring("Metadata Input", "Enter Solvent:")
    metadata["hemoglobin_concentration_mg_mL"] = simpledialog.askfloat("Metadata Input", "Enter Hemoglobin Concentration (mg/mL):")
    metadata["microsphere_concentration_uL_mL"] = simpledialog.askfloat("Metadata Input", "Enter Microsphere Concentration (uL/mL):")
    metadata["yeast_stock_added_uL_mL"] = simpledialog.askfloat("Metadata Input", "Enter Yeast Stock Added (uL/mL):")
    metadata["yeast_concentration_mg_mL"] = simpledialog.askfloat("Metadata Input", "Enter Yeast Concentration (mg/mL):")

    return metadata

def migrate_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create new tables if they don't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dissolved_oxygen_study_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            start_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            sample_name TEXT NOT NULL,
            solvent TEXT DEFAULT 'water',
            hemoglobin_concentration_mg_mL REAL DEFAULT NULL,
            microsphere_concentration_uL_mL REAL DEFAULT NULL,
            yeast_stock_added_uL_mL REAL DEFAULT NULL,
            yeast_concentration_mg_mL REAL DEFAULT NULL,
            UNIQUE(id)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dissolved_oxygen_records (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sample_name TEXT NOT NULL,
            sample_id INTEGER NOT NULL,
            time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            dissolved_oxygen REAL DEFAULT NULL,
            nanoamperes REAL DEFAULT NULL,
            temperature REAL DEFAULT NULL,
            FOREIGN KEY(sample_name, sample_id) REFERENCES dissolved_oxygen_study_table(sample_name, id)
        )
    ''')
    conn.commit()

    # Get list of old tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND tbl_name NOT LIKE 'sql%'")
    tables = [row[0] for row in cursor.fetchall()]

    for table in tables:
        if table in ('dissolved_oxygen_study_table', 'dissolved_oxygen_records'):
            continue  # Skip new tables

        # Extract metadata from user
        metadata = get_metadata_from_user(table)
        cursor.execute('''
            INSERT INTO dissolved_oxygen_study_table
            (sample_name, solvent, hemoglobin_concentration_mg_mL, microsphere_concentration_uL_mL,
            yeast_stock_added_uL_mL, yeast_concentration_mg_mL)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (metadata["sample_name"], metadata["solvent"], metadata["hemoglobin_concentration_mg_mL"],
              metadata["microsphere_concentration_uL_mL"], metadata["yeast_stock_added_uL_mL"],
              metadata["yeast_concentration_mg_mL"]))

        sample_id = cursor.lastrowid  # Get inserted row ID

        # Copy data into dissolved_oxygen_records
        cursor.execute(f"SELECT id, time, dissolved_oxygen, nanoamperes, temperature FROM {table}")
        rows = cursor.fetchall()

        for row in rows:
            _, time, dissolved_oxygen, nanoamperes, temperature = row
            cursor.execute('''
                INSERT INTO dissolved_oxygen_records
                (sample_name, sample_id, time, dissolved_oxygen, nanoamperes, temperature)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (metadata["sample_name"], sample_id, time, dissolved_oxygen, nanoamperes, temperature))

        conn.commit()
        print(f"Migrated {len(rows)} records from {table}.")

    # Confirm and drop old tables
    confirm = simpledialog.askstring("Confirmation", "Do you want to drop the old tables? (yes/no):")
    if confirm and confirm.lower() == "yes":
        for table in tables:
            if table not in ('dissolved_oxygen_study_table', 'dissolved_oxygen_records'):
                cursor.execute(f"DROP TABLE {table}")
                print(f"Dropped table {table}.")
        conn.commit()

    conn.close()
    print("Database migration complete.")

In [2]:
root = tk.Tk()
root.withdraw()  # Hide the main window
db_path = filedialog.askopenfilename(title="Select SQLite Database", filetypes=[("SQLite Database", "*.sqlite;*.db")])
if db_path:
    migrate_database(db_path)

Migrated 1604 records from water_henry.
Migrated 4200 records from water_yeast_25ul_of_250mg_mL_yeast.
Migrated 2200 records from water_yeast_100ul_of_250mg_mL_yeast.
Migrated 125 records from phantom13_14_henry.
Migrated 122 records from phantom12.
Migrated 127 records from phantom9.
Migrated 122 records from phantom6_henry.
Migrated 631 records from phantom3_henry.
Database migration complete.
